# Week 2

## Word representation

* Problems with the standard 1-hot presentation of words: model finds it hard to learn relationships between words (king is to queen as man is to woman):
  * Inner product between any 2 one-hot vectors is 0.
  * Euclidean distance between any pair of vectors is the same.
* Would be better to learn a set of features for each words that help to categorise it (eg gender, royal-ness, age, fruit-ness etc).
* Embedding basically learns relevant features (usually not easy to interpret), generating a 300 (or whatever) dimensions vector for each word.

## Using word embeddings

* Named entity recogition example: what happens if you come across a set of words that your model hasn't learning in the standard RNN approach? Enter embedding.
* Embedding can be trained on much larger corpuses of text (1B - 100B words is not uncommon).
* Embedding allows for transfer learning: models using pretrained embeddings can be trained on smaller corpuses (à la ImageNet models in the image classificatino word).
  * Embeddings can be finetuned with new data (if dataset if big enough).
* Embeddings have been useful for: entity recognition, text summarisation, co-reference resolution, parsing.
* Less useful for: language modeling, machine translation.

### Properties of word embeddings

* Word embeddings can help with "analogy reasoning".
  * Answering the question: "Man is to woman as king is to ___ ?"
* With a 4 dimension embedding for man and woman, with a feature for gender, royalness, age and foodness, you could subtract them from each other:

    ```
    e_man - e_woman = [-2, 0, 0, 0]
    ```
    
  If you did the same from king and queen, you may end up with a similar result:
   
    ```
    e_king - e_queen = [-2, 0, 0, 0]
    ```
   
  * What it captures is that the main different between the two word sets is the gender.
  * Ideas first published in paper [Linguistic Regularities in Continuous Space Word Representations](https://www.aclweb.org/anthology/N13-1090).
* Formally, you'd aim to find the $e_{w} that satisfies that expression:
   
   $e_{man} - e_{woman} \approx e_{king} - e_{w}$
   
   * Find word $e_{w}$ that maximises this the similarity expression (ie has the highest degree of similiarity): $\text{sim}(e_{w}, e_{king} - e_{man} + e_{woman})$
   
* Similarity function: cosine similarity:
  * $sim(u, v) = \frac{u^{t}v}{||u||_2 ||v||_1}$
    * Without the denominator, it's basically the inner product: if they are similar, product will be large.
    * With the denominator, normalises result between -1 and 1. Very distant = -1, very similar = 1.
  * Could also use squared distance: $||u-v||^2$ (since it's a measure of dissimilarity, you should take the negative to maximise.

### Embedding matrix



* Embedding matrix will be a `(300, 10000)` ($E$) dimension matrix (if you have a 10k vocab and 300 latent factors).
* You could look up a word value in the matrix using a `(10000, 1)` ($o_j$) one-hot encoded matrix and doing a dot product against the embedding matrix to get the 300 latent factors for the word:
  $E * o_j = e_j$
* In practise you'd generally just lookup the embedding value of a word using a dictionary lookup, but it's easier to represent mathematically as a dot product.

### Learning word embeddings

* In the early days of learning word embeddings, the algorithms were quite complicated but over time, researchers discovered simpler ways to do it.
* If you started with a sentence and took away the last word:
  
    "I     want   a    glass   of     orange   ___."
     4343  9665   1    3852    6161   6257    
     
    * You would then lookup the embedding for each word:
     * "I"       $o_{4343} \longrightarrow E \longrightarrow e_{4343}$
     * "want"    $o_{9665} \longrightarrow E \longrightarrow e_{9665}$
     * "a"       $o_{1}    \longrightarrow E \longrightarrow e_{1}$
     * "glass"   $o_{3852} \longrightarrow E \longrightarrow e_{3852}$
     * "of"      $o_{3852} \longrightarrow E \longrightarrow e_{3852}$
     * "orange"  $o_{6257} \longrightarrow E \longrightarrow e_{6257}$
    * Then pass all the values into a neural network, which feeds into a softmax layer. Softmax classifies against the 10k values and outputs a single word.
    * The number of weights in the final layer is dependant on the size of the "fixed historic window" hyperparam. In other words, you could choose to take the last n words to predict the output.
    * The number of weights would be 4 times the number of latent factors, in the above example: `4 * 300`.
    * Use backprop to find the ideal word embeddings for the task.
    
* More complex algorithms:
  * Given a sentence: "I want a glass of orange juice to go along with my cereal." try to pick one of the middle words.
    * You might feed into the neural net 4 words on the left & right.
    * You could also just pass in a single last word.
    * You could also just take a nearby 1 word (glass, cereal etc) - this is called a skip gram model.

### Word2Vec

* "A simpler and computational more effective way to learn embeddings".
* Given sentence: "I want a glass of orange juice to go along with my cereal", you find a context -> target pair randomly (called a 'skip gram'), ie `orange` and `juice`, or `orange` and `glass` or `orange` and `my`.
* Probably won't do well on this problem, but you aren't looking for success on this problem, you want to learn a good embedding.
* $O_c \longrightarrow E \longrightarrow e_c \longrightarrow softmax \longrightarrow \hat{y} $
* Softmax: $p(t|c) = \frac{e^{\theta_{t}^{T}e_c}}{\sum\limits_{j=1}^{10000} e^{\theta_{j}^{T}e_c}}$
* $L(\hat{y}, y) = -\sum\limits_{i=1}^{10000} y_i \log \hat{y}_i $

* Problems with the model:
  * Everytime you want to calculate output, you need to sum over entire dictionary size (10k +).
* Solution: hierarchical softmax classifier.
   * Instead of trying to categorise something into 10k categories in one go, you have a classifier that says if something is in the first half or second half. Then another for the half of that and so on, until you have the correct answer.

### Negative Sampling

* Modified learning problem that solves the issue of softmax being slow to compute.
* New problem: given a pair of words like "orange" "juice" is it a context pair?
  * Orange -> juice: target.
* Then take first word and pick another random word that's not a context pair.
  * Orange -> king: not target.
  * Orange -> book: not target.
  * Up to k examples.
* k = 
  * 5-20 for smaller datasets.
  * 2-5 for larger datasets.
* Want to build model that will output 1 if it's a correct context pair and 0 if not (hence negative sampling).
  * Define: `context` (aka "orange") as $c$, `word` (aka "juice" or "queen") as $t$ and `target` (1 if target, 0 if not) as $y$.
      * Then, create a logistic regression model probably y is target, given the context and the target pair: $P(y=1 | c, t) = \sigma(\theta_{t}^{T}e_{c})$.
        * Create a parameter _vector_ $\theta_{t}$ and do a dot product with the context embedding vector to predict the probability that y = 1
      * Should have a 10k (or vocab size) output where each element is the probability of y.
        * This gives 10k binary classification problems where each is relatively easy to do.
        * Plus you are only training k (the number of bad examples) + 1 at a time.
* May need to consider how to sample the negative words;
  * Could sample according to how often a word appears (emperical frequency).
    * Problem: end up with high representation of common words: "the", "of", "and" etc.
  * Could sample uniformly at random, but that's also non representative of the distribution.
* The authors of the paper found a different method worked best:
  $P(w_i) = \frac{f(wi)^{\frac{3}{4}}}{\sum\limits_{j=1}^{10000} f(w_j)^{3/4}}$
    * If $f(w_i)$ is the probability that a word that a exists in the distribution, you take it by the power of 3/4, which is between the extreme of taking the observed distribtion and the uniform distribution.
    * The denominator is to normalise it.
* Lots of open-source implementations and pretrained word vectors available for skip gram models.

### GloVe word vectors

* "Global vectors for word representation."
* Not used as much as word2vec, but has some fans because of how simple it is.
* Can picking 2 pairs $c$, $t$ and figure out how many times they appear within the same context: $X_{ij} = \text{# times i (t) appears in the context to j (c)}$
* If context and target are defined as whether or not they appear within + or - 10 spots of each other, relationship would be symmetrical: $X{ij} = X{ji}$.
* Glove model minimises the sum of difference between embedding and log Xij squared: $\sum\limits_{i=1}^{10000} \sum\limits_{j=1}^{10000} (\theta_{i}^{T} e_j - log X_{ij})^2$
  * If $X_{ij} = 0$ then you don't want to try to calculate the log term, since it's negative infinity. Instead, add a weighting term that sets result to 0:
    $\sum\limits_{i=1}^{10000} \sum\limits_{j=1}^{10000} f(X_{ij})(\theta_{i}^{T} e_j - log X_{ij})^2 \\ f(X_{ij}) = 0 \text{ if } X_{ij} = 0$
    * Other use case for $f(X_{ij})$ is to add a weighting factor that gives more weight to common words, but not crazy big but also ensures weight for less common words.
    * Various heuristics for choosing weighting function.
  * Roles of $\theta$ and $e_j$ are symmetric (they place same role and can be reversed).
    * So, you can train the algorithm by initilsing $\theta$ and $e_j$ randomly, then when done, can calculate $e_w^{(final)} = \frac{e_w + \theta_w}{2}$, basically taking the average of both params.
* GloVe came later but was essentially a simplification of word embedding creation algorithms.
* Can be hard to interpret features directly but can do the analogy stuff covered earlier.

### Sentiment classification problem

* Input: "The dessert is excellent", Output: number of stars (eg 5).
* Can you try a system to map from x to y, could use to monitor comments on restaurant.
* One approach: Sum the embedding values, then average the results and feed into a softmax layer with 5 outputs.
  * Average allows you to deal with reviews that are short or long.
  * Problem with that is, reviews like "Completely lacking in good taste, good service and good ambience", would probably be positive due to the number of occurences of "good" (a positive word).
* More sophisticated algorithm: use a many-to-one RNN which takes the embedding values in at each step.

### The problem of bias in word embeddings

* Paper [Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings](https://arxiv.org/abs/1607.06520) discovered that a learned embedding learned: "man is to computer programmer as woman is to homemaker" as well as "father is to doctor as mother is to nurse".
* Word embeddings can reflect gender, ethnicity, age, sexual orientation and other biases of the text used to train the model.
  * "We have better ideas for quickly reducing bias in AI than in the human race".
1. Identify bias direction of words that should have bias:
  * $e_{he}$ - $e_{she}$
  * $e_{male}$ - $e_{female}$
  * $e_{grandfather}$ - $e_{grandmother}$
  * Then take the average.
2. Neutralize: for every word that is not definitional, project to get rid of bias by centreing results.
3. Equalize pairs: try to move words that should have bias to equal distances away from words that have been neutralized.
  * Some words like "beard" may be gender definitional.
* Paper authors trained a classifier to find words that need to be neutralized.
  * Found that number of words that you want to equalize is usually quite small.